In [1]:
import pandas as pd
import numpy as np
import pyomo.environ as pyo
from numpy.random import normal
import qmcpy as qp

In [2]:
#  weight parameters are uncertain with normal distribution and known mean, standard deviation
df = pd.read_csv('knapsack_dataset_uncertain_weight.csv')  
print(df)

     name  value  weight_mean  weight_std
0    gold    100          4.0         0.1
1  silver     90          2.8         0.4
2  bronze     70          2.2         0.3


In [3]:
knapsack_capacity = 6

In [4]:
number_of_scenarios = 128  # must be a power of 2
g = qp.Gaussian(qp.Sobol(len(df),seed=7),mean=df['weight_mean'],covariance=df['weight_std'])
scenarios = g.gen_samples(number_of_scenarios)

In [5]:
def init_weight_param(model, i, j):
    return scenarios[i, j]

In [6]:
def init_value_param(model, i):
    return df.loc[i, 'value']

In [7]:
def weight_limit_rule(m, j):
    return sum(m.w[j, i] * m.x[i] for i in range(len(df))) <= knapsack_capacity

In [8]:
model = pyo.ConcreteModel()

In [9]:
# indices definition
model.ind_items = pyo.RangeSet(0, len(df)-1)
model.ind_scenarios = pyo.RangeSet(0, number_of_scenarios-1)

In [10]:
# variable definition
model.x = pyo.Var(model.ind_items, domain=pyo.Binary)

In [11]:
# parameter defnition
model.w = pyo.Param(model.ind_scenarios, model.ind_items,
                    initialize=init_weight_param, within=pyo.PositiveReals)
model.v = pyo.Param(model.ind_items, initialize=init_value_param, within=pyo.PositiveReals)

In [12]:
# objective function definition
model.obj = pyo.Objective(expr = sum((1/number_of_scenarios) * sum(model.v[i] * model.x[i] for i in model.ind_items)
                          for _ in model.ind_scenarios), sense=pyo.maximize)

In [13]:
# constraints definition
model.weight_limit_constraints = pyo.Constraint(model.ind_scenarios, rule=weight_limit_rule)

In [14]:
# solving the model
opt = pyo.SolverFactory('gurobi')
result = opt.solve(model)

In [15]:
model.display()

Model unknown

  Variables:
    x : Size=3, Index=ind_items
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0 :   1.0 :     1 : False : False : Binary
          1 :     0 :   0.0 :     1 : False : False : Binary
          2 :     0 :   0.0 :     1 : False : False : Binary

  Objectives:
    obj : Size=1, Index=None, Active=True
        Key  : Active : Value
        None :   True : 100.0

  Constraints:
    weight_limit_constraints : Size=128
        Key : Lower : Body               : Upper
          0 :  None :  4.140143105285576 :   6.0
          1 :  None : 3.8695010924640423 :   6.0
          2 :  None :  3.606721392078507 :   6.0
          3 :  None :  4.413564993385888 :   6.0
          4 :  None : 4.3052096289792114 :   6.0
          5 :  None : 3.6902823225812407 :   6.0
          6 :  None :  3.917732863783074 :   6.0
          7 :  None :  4.079559242534682 :   6.0
          8 :  None : 3.8001190018788105 :   6.0
          9 :  None :  4.21197296